In [ ]:
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as pl
# Set working directory
os.chdir("P:/Tolulope/Manuscript/Yuan Analysis")
adata = sc.read_h5ad('combined data.h5ad')
adata
adata.X
adata.obs.groupby('Sample').count()
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata
adata.obs.head()
scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             categorical_covariate_keys=["Sample"],
                             continuous_covariate_keys=['pct_counts_mt', 'total_counts'])
model = scvi.model.SCVI(adata)
model.train() #may take a while without GPU
adata.obsm['X_scVI'] = model.get_latent_representation()
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep = 'X_scVI')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution = 0.8)
sc.pl.umap(adata, color = ['leiden', 'Sample'], frameon = False)
# Customize UMAP plot
sc.set_figure_params(figsize=(10, 10))
img_tag = "_color"  # Specify your image tag here
sc.pl.umap(adata, color=['Sample', 'leiden'], legend_loc='on data', legend_fontsize=12, legend_fontoutline=2, frameon=False, size=35, title='Clustering of Cells', save=f'umap_plot after integration sample and cell type scv{img_tag}.png')
pl.show()
# Find markers/ label cell types

sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, save=f'rankgenese{img_tag}.png')
markers = sc.get.rank_genes_groups_df(adata, None)
markers = markers[(markers.pvals_adj < 0.05) & (markers.logfoldchanges > .5)]
markers
markers_scvi = model.differential_expression(groupby = 'leiden')
markers_scvi
markers_scvi = markers_scvi[(markers_scvi['is_de_fdr_0.05']) & (markers_scvi.lfc_mean > .5)]
markers_scvi
sc.pl.umap(adata, color = ['Pax7', 'Esam'], frameon = False, layer = 'scvi_normalized', vmax = 5)
#, layer = 'scvi_normalized'
markers[markers.names =='Pecam1']  


In [ ]:
cell_type = {
    "0": "FAPs I",
    "1": "Residence Macrophages/APCs I",
    "2": "Neutrophils",
    "3": "Monocyte/Macrophages",
    "4": "M2-like macrophages",
    "5": "MuSCs",
    "6": "Mature skeletal muscle I",
    "7": "Endothelial cells I",
    "8": "Myo-Fibroblasts",
    "9": "FAPs II",
    "10": "Endothelial cells II",
    "11": "Lipid-Laden Macrophages",
    "12": "Mature skeletal muscle II",
    "13": "Dendritic cells",
    "14": "Residence Macrophages/APCs II",
    "15": "T cells/NKCs",
    "16": "Fusion-Competent Myoblasts",
    "17": "Proliferation",
    "18": "B cells",
    "19": " Migratory immune cells",
    "20": "Vascular APCs",
   
        
}
# Now map the 'leiden_R' column to 'cell type'
adata.obs['cell type'] = adata.obs['leiden'].map(cell_type)


In [ ]:
#UMAP plot
sc.set_figure_params(figsize=(10, 10))
img_tag = "_color"  # Specify your image tag here
sc.pl.umap(adata, color=['cell type'],  legend_fontsize=12, legend_fontoutline=2, frameon=True, size=80, title='Clustering of Cells', save=f'umap_plot{img_tag}.png')
pl.show()

In [ ]:
adata.uns['scvi_markers'] = markers_scvi
adata.uns['markers'] = markers
adata.write_h5ad('integratedssssss.h5ad')
model.save('model.model')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import seaborn as sns
from matplotlib import rc_context
import matplotlib as mpl
import pandas as pd
import os
import scanpy as sc
from scipy.sparse import csr_matrix
import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [ ]:
# Set working directory and read the AnnData object
os.chdir("P:/Tolulope/Manuscript/Yuan Analysis")
# Set the output directory for saving plots
sc.settings.figdir = "FIGURE_1"

adata = sc.read_h5ad('integratedssssss.h5ad')
adata

In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define custom color palette for cell types
celltypePalette = {
    "FAPs I": '#9467bd',
    "Residence Macrophages/APCs I": '#17becf',
    "Neutrophils": '#c5b0d5',
    "Monocyte/Macrophages": '#8c564b',
    "M2-like macrophages": '#1f77b4',
    "Satellite Cells": '#2ca02c',
    "Mature skeletal muscle I": '#ff7f0e',
    "Endothelial cells I": '#d62728',
    "Myo-Fibroblasts": '#9edae5',
    "FAPs II": '#008080',
    "Endothelial cells II": '#bcbd22',
    "Lipid-Laden Macrophages": '#ffcc00',
    "Mature skeletal muscle II": '#ffbb78',
    "Dendritic cells": '#dbdb8d',
    "Residence Macrophages/APCs II": '#FF6347',
    "T cells/NKCs": '#c7c7c7',
    "Fusion-Competent Myoblasts": '#7FFF00',
    "Proliferation": '#FF69B4',
    "B cells": '#00BFFF',
    " Migratory immune cells": '#E2725B',
    "Vascular APCs": '#c49c94'  
}

# Set figure parameters
sc.set_figure_params(figsize=(10, 10), dpi=300, dpi_save=300, fontsize=12)

# Plot base UMAP with custom colors and no legend (set legend_loc='none')
sc.pl.umap(adata, color='cell type', palette=celltypePalette, frameon=True, size=40, show=False, legend_loc=None)

# Add text labels at the centroid of each cell type
umap = adata.obsm['X_umap']
cell_types = adata.obs['cell type']

# Compute centroids for each cell type
label_coords = pd.DataFrame(umap, columns=['x', 'y'])
label_coords['cell type'] = cell_types
centroids = label_coords.groupby('cell type')[['x', 'y']].median()

# Add labels to the plot
for cell_type, (x, y) in centroids.iterrows():
    plt.text(x, y, cell_type,
             fontsize=9,  # Adjusted for better visibility
             weight='bold',
             color=celltypePalette.get(cell_type, 'black'),
             ha='center',
             va='center')

# Title and clean layout
plt.title('Clustering of Cells (UMAP)', fontsize=16)
plt.axis('off')  # Hide axes

# Optional: Adjust axis limits to ensure labels are visible
plt.xlim([np.min(umap[:, 0]) - 1, np.max(umap[:, 0]) + 1])
plt.ylim([np.min(umap[:, 1]) - 1, np.max(umap[:, 1]) + 1])

# Save and show the plot
plt.tight_layout(pad=2)
plt.savefig('umap_celltype_locked_no_legend.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:

# Specify the gene names for plotting
genes_for_violin = ['Pax7', 'Vcam1', 'Mymk', 'Mymx', 'Esam', 'Pecam1','Cthrc1', 'Col1a1', 'Dcn', 'Pdgfra', 'Cd163', 'Folr2','Lyve1','Cd74', 'H2-DMb1', 
                    'H2-Ab1', 'H2-Aa', 'Cd79a', 'Igkc', 'Arg1', 'Cxcl3', 'Cd3e', 
                    'Nkg7', 'S100a8', 'S100a9', 'Birc5',  'Top2a', 'Alcam', 'Ttn', 'Ckm','Gpnmb', 'Fabp5']

# Calculate the number of rows needed based on the number of genes
num_rows = len(genes_for_violin)

# Create subplots
fig, axs = plt.subplots(nrows=num_rows, ncols=1, figsize=(10, 35))

# Ensure axs is iterable when num_rows = 1
if num_rows == 1:
    axs = [axs]

# Loop through genes and plot each one
for i, gene in enumerate(genes_for_violin):
    sc.pl.violin(adata, gene, groupby='cell type',  split=False, stripplot=False, rotation=90, 
                 use_raw=True, scale="width", ax=axs[i], show=False, fill=True, inner="box")
    
    # Hide x-axis labels for all but the last plot
    if i != num_rows - 1:
        axs[i].set_xticklabels([])
        axs[i].set_xlabel('')
    else:
        axs[i].set_xlabel('Cell Type', fontsize=12)
        axs[i].tick_params(axis='x', labelrotation=90)  # Rotate x-axis labels
    
    # Access and customize the inner box color
    for patch in axs[i].collections:
        if isinstance(patch, plt.Polygon):  # Check for the inner box (which is a Polygon)
            patch.set_edgecolor('black')  # Set the edge color to black
            patch.set_facecolor('black')  # Set the fill color to black

# Adjust spacing
plt.tight_layout()

# Save the figure
output_path = os.path.join(sc.settings.figdir, "Integratedviolinmarker_plot.png")
plt.savefig(output_path, dpi=300)

# Display all plots
plt.show()


In [ ]:
adata = sc.read("integratedssssss.h5ad")
adata_concat=adata.copy()
# Now, check the raw attribute of the concatenated data
print(adata_concat.raw)  # Print the raw object
print(adata_concat.raw.var.shape)  # Print the shape of raw gene data
ad_filtered = adata_concat[((adata_concat.raw[: , "Gfp"].X.toarray().flatten() ==0) & (adata_concat.raw[: , "Pax7"].X.toarray().flatten() ==0)), :].copy()
#ad_filtered = adata_concat[(adata_concat.raw[: , "Gfp"].X.toarray().flatten() ==0) , :].copy()
print(ad_filtered)
sc.tl.embedding_density(ad_filtered, groupby='Sample')

In [ ]:
import matplotlib.pyplot as pl
img_tag = "_color"  # Specify your image tag here
# Set figure size
pl.rcParams['figure.figsize'] = 8, 8
sc.pl.embedding_density(ad_filtered, groupby='Sample', ncols=4, frameon=False, save=f"umap_densityplot2{img_tag}.png")


In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

# Set figure size for consistent styling
plt.rcParams['figure.figsize'] = (8, 8)

# Set font to Arial for Nature publication standard
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 10  # Adjust font size for publication

# Define a variable for image tag for flexibility in saving
image_tag = "_color"  # Specify your image tag here

# Create a UMAP density plot with the necessary adjustments
sc.pl.embedding_density(
    ad_filtered,               # Data object (AnnData)
    groupby='Sample',          # Grouping variable for color coding (e.g., different conditions)
    ncols=4,                   # Number of columns for arranging subplots
    frameon=False,             # Turn off frame around plot
    color_map='viridis',       # Choose a perceptually uniform and publication-friendly color map
    legend_loc='right margin', # Position the legend appropriately
    save=f"umap_densityplot2{image_tag}.png",  # Save plot with a descriptive name
)

# Optional: Add additional annotations or title for the figure
plt.suptitle('UMAP Density Plot of Samples', fontsize=14)
plt.xlabel('UMAP 1', fontsize=12)  # Label x-axis
plt.ylabel('UMAP 2', fontsize=14)  # Label y-axis

# Ensure the plot is clear and professional before saving
plt.tight_layout()  # Adjust layout to prevent overlapping labels

# Save the figure with high resolution
plt.savefig(f"umap_densityplot2{image_tag}.png", dpi=300)


In [ ]:
ad_filtered.obs[['umap_density_Sample','leiden','Sample']].to_csv('density_by_cluster_filteredscvitools1.csv')

In [ ]:
# Set figure size
pl.rcParams['figure.figsize'] = 1.5, 1.5

# Create the UMAP plot and save it
sc.pl.umap(adata_concat, color='Gfp', color_map='magma_r', save=f"umap_Gfp_{img_tag}.png")

In [ ]:
# Set figure size
pl.rcParams['figure.figsize'] = 1.5, 1.5

# Create the UMAP plot and save it
sc.pl.umap(adata_concat, color='Pax7', color_map='magma_r', save=f"umap_Pax7_{img_tag}.png")

In [ ]:
# Set figure size
pl.rcParams['figure.figsize'] = 3, 2

# Scatter plot of Pax7 vs Gfp and save it
sc.pl.scatter(adata_concat, x="Pax7", y="Gfp", size=10, save=f"umap_correlation_{img_tag}.png")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define parameters
genes = ['Dnm1', 'Cd209a', 'Itga1', 'Scarb1','Cdc42']
samples = ['YSham_GFP', 'YSham_noGFP', 'ASham_GFP', 'ASham_noGFP', 'YOV_GFP', 'YOV_noGFP', 'AOV_GFP', 'AOV_noGFP']
cell_types = ["Residence Macrophages/APCs I", "Monocyte/Macrophages", "M2-like macrophages", "Lipid-Laden Macrophages"]

# Subset the AnnData object
adata_sub = adata[adata.obs['cell type'].isin(cell_types) & adata.obs['Sample'].isin(samples)].copy()

# Prepare long-format data for plotting
plot_data = []
for gene in genes:
    for ct in cell_types:
        for sample in samples:
            adata_temp = adata_sub[
                (adata_sub.obs['cell type'] == ct) & (adata_sub.obs['Sample'] == sample), gene
            ]
            expr = adata_temp.X.toarray().flatten() if hasattr(adata_temp.X, "toarray") else adata_temp.X.flatten()
            pct_expr = np.mean(expr > 0) * 100
            mean_expr = np.mean(expr)
            plot_data.append({
                'Gene': gene,
                'Sample': sample,
                'Cell Type': ct,
                '% Expressing': pct_expr,
                'Mean Expression': mean_expr
            })

df = pd.DataFrame(plot_data)

# Order the categories for consistent plotting
df = df[df['Sample'].isin(samples)].copy()
df['Sample'] = pd.Categorical(df['Sample'], categories=samples, ordered=True)
df['Gene'] = pd.Categorical(df['Gene'], categories=genes, ordered=True)
df['Cell Type'] = pd.Categorical(df['Cell Type'], categories=cell_types, ordered=True)

# Set seaborn styling
sns.set(style="white", context="talk", font_scale=1.1)

# Plotting
n_celltypes = len(cell_types)
fig, axes = plt.subplots(1, n_celltypes, figsize=(4 * n_celltypes, 6), sharey=True)

if n_celltypes == 1:
    axes = [axes]

for ax, cell_type in zip(axes, cell_types):
    data_sub = df[df['Cell Type'] == cell_type]

    scatter = ax.scatter(
        x=data_sub['Gene'],
        y=data_sub['Sample'],
        s=data_sub['% Expressing'] * 3,
        c=data_sub['Mean Expression'],
        cmap='Reds',
        edgecolors='black',
        linewidth=0.7,
        alpha=0.85,
        marker='o'
    )

    ax.set_title(cell_type, fontsize=14)
    ax.set_xlabel('')
    ax.set_ylabel('Condition' if ax == axes[0] else '')
    ax.set_xticks(range(len(genes)))
    ax.set_xticklabels(genes, rotation=45, ha='right')
    ax.set_yticks(range(len(samples)))
    ax.set_yticklabels(samples)

    # Remove gridlines
    ax.grid(False)

    # Customize plot border (spines)
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)

# Adjust layout to make room for colorbar
plt.tight_layout(rect=[0, 0, 0.88, 1])

# Add compact colorbar on top-right
cbar_ax = fig.add_axes([0.91, 0.75, 0.015, 0.2])  # [left, bottom, width, height]
cbar = fig.colorbar(scatter, cax=cbar_ax, orientation='vertical')
cbar.set_label('Mean Expression', rotation=270, labelpad=12, fontsize=12)

# Global title
plt.suptitle("Expression of Phagocytic Markers in Macrophage Subsets", fontsize=16, y=1.03)

# Save plots
plt.savefig("PhagocyticMarker_DotPlot1.pdf", dpi=600, bbox_inches='tight')
plt.savefig("PhagocyticMarker_DotPlot1.png", dpi=600, bbox_inches='tight')

plt.show()
